In [2]:
import numpy as np
import torch 
import sys
sys.path.append("../")
import scipy

import pandas as pd
import matplotlib.pyplot as plt
# import utils_BayTIDE as utils
from scipy.io import loadmat

In [4]:
# impory solar.txt as np.array
tensor = np.loadtxt('/data/fang/BayTIDE_data/raw/solar.txt', delimiter=',')


tensor = tensor.T
print(tensor.shape)

# check nan, max and min and mean
print(np.isnan(tensor).any())
print(np.nanmax(tensor))
print(np.nanmin(tensor))
print(np.nanmean(tensor))



(137, 52560)
False
88.9
0.0
6.35265222088902


In [6]:
sub_tensor = tensor

data_save = {}
data_save['ndims'] = sub_tensor.shape

data_save['raw_data'] = sub_tensor

data_save['data'] = []

# sorted unique timestamps, set as (normalized to 0-1 ) regular time intervals at current stage
data_save['time_uni'] = np.linspace(0,1,sub_tensor.shape[1])

def generate_random_mask( shape, drop_rate=0.2, valid_rate=0.1):
    """
    train_ratio: 1-valid_rate-drop_rate
    test_ratio: drop_rate
    valid_ratio: valid_rate
    """
    N,T = shape

    train_T = int(T*(1-valid_rate-drop_rate))

    T_index = np.arange(T)

    np.random.shuffle(T_index)

    mask_train = np.zeros((N,T))
    mask_test = np.zeros((N,T))
    mask_valid = np.zeros((N,T))

    mask_train[:,T_index[:train_T]] = 1
    mask_test[:,T_index[train_T:train_T+int(T*drop_rate)]] = 1
    mask_valid[:,T_index[train_T+int(T*drop_rate):]] = 1

    return mask_train, mask_test, mask_valid

fold = 2
drop_rate = 0.4
valid_rate = 0.1

for i in range(fold):
    mask_train, mask_test, mask_valid = generate_random_mask(sub_tensor.shape, drop_rate, valid_rate)
    data_save['data'].append({'mask_train':mask_train, 'mask_test':mask_test, 'mask_valid':mask_valid})

file_name = '/data/fang/BayTIDE_data/solar_impute'+'_r_%.1f'%(drop_rate)+'_irregulate.npy'
np.save(file_name, data_save)


In [8]:
mask_train.sum()/mask_train.size

0.49999430612494306